# 1. Import the necessary libraries for you to be able to query 4square API

In [66]:
import requests
import json
import pandas as pd
from getpass import getpass

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [80]:
API_Key = getpass.getpass("Enter your secret token: ")

Enter your secret token:  ········


In [92]:
#Base url for Foursquare API 
url = "https://api.foursquare.com/v3/places/search"
headers = {"accept": "application/json",
           "Authorization": API_Key}
#Parameters for the search
params = {
    "query": "coffee",
    "ll": "52.6586,13.3496", # cordinate of Ironhack in Berlin
    "limit": 10
}
r = requests.get(url, headers = headers, params = params)
print(f"Status code: {r.status_code}")
print(f"Request URL: {r.url}")
print(f"Response Text: {r.text}")

Status code: 200
Request URL: https://api.foursquare.com/v3/places/search?query=coffee&ll=52.6586%2C13.3496&limit=10
Response Text: {"results":[{"fsq_id":"4de78ec4d4c0086a950caab2","categories":[{"id":13027,"name":"Bistro","short_name":"Bistro","plural_name":"Bistros","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}},{"id":13034,"name":"Café","short_name":"Café","plural_name":"Cafés","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/cafe_","suffix":".png"}},{"id":13035,"name":"Coffee Shop","short_name":"Coffee Shop","plural_name":"Coffee Shops","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":4595,"geocodes":{"drop_off":{"latitude":52.665873,"longitude":13.282594},"main":{"latitude":52.666083,"longitude":13.282536},"roof":{"latitude":52.666083,"longitude":13.282536}},"link":"/v3/places/4de78ec4d4c0086a950caab2","location":{"address":"S

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [126]:
#Base url for Foursquare API 
url = "https://api.foursquare.com/v3/places/search"
headers = {"accept": "application/json",
           "Authorization": API_Key}
#Parameters for the search
params = {
    "query": "coffee",
    "ll": "52.6586,13.3496", # cordinate of Ironhack in Berlin
    "limit": 5
}
r = requests.get(url, headers = headers, params = params)
print(f"Status code: {r.status_code}")
print(f"Request URL: {r.url}")
print(f"Response Text: {r.text}")

Status code: 200
Request URL: https://api.foursquare.com/v3/places/search?query=coffee&ll=52.6586%2C13.3496&limit=5
Response Text: {"results":[{"fsq_id":"4de78ec4d4c0086a950caab2","categories":[{"id":13027,"name":"Bistro","short_name":"Bistro","plural_name":"Bistros","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}},{"id":13034,"name":"Café","short_name":"Café","plural_name":"Cafés","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/cafe_","suffix":".png"}},{"id":13035,"name":"Coffee Shop","short_name":"Coffee Shop","plural_name":"Coffee Shops","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":4595,"geocodes":{"drop_off":{"latitude":52.665873,"longitude":13.282594},"main":{"latitude":52.666083,"longitude":13.282536},"roof":{"latitude":52.666083,"longitude":13.282536}},"link":"/v3/places/4de78ec4d4c0086a950caab2","location":{"address":"Sc

In [148]:
#parse the Json response
data = r.json()
results = data.get("results", [])

#create a list for places 
places = []
for place in results:
    name = place.get("name", "N/A")
    address = place.get("location", {}).get("formatted_address", "N/A")
    distance = place.get("distance", "N/A")  # Distance from the location
    latitude = place.get("geocodes", {}).get("main", {}).get("latitude", "N/A")
    longitude = place.get("geocodes", {}).get("main", {}).get("longitude", "N/A")
    places.append({"Name": name, "Address": address, "Distance (m)": distance, "Latitude": latitude, "Longitude": longitude})
closest_coffee = pd.DataFrame(places)
closest_coffee.head()

,Name,Address,Distance (m),Latitude,Longitude
0,Kaffeehaus Morgenrot,"Scharfschwerdtstr. 1, 16540 Hohen Neuendorf",4595,52.666083,13.282536
1,Kaffeehaus Birkenwerder,"Clara-Zetkin-Str. 19a, 16547 Birkenwerder",5407,52.688385,13.286188
2,Morgenrot,"Scharfschwerdtstr. 1, 16540 Hohen Neuendorf",4328,52.665162,13.286295
3,Coffee Circle,"Lindower Str. 18, 13347 Berlin",12815,52.543846,13.367767
4,Kajumi,"Dunckerstr. 73, 10437 Berlin",13613,52.544229,13.421858


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [241]:
def foursquare_places (venue, coordinates):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {"accept": "application/json",
               "Authorization": API_Key}
    #Parameters for the search
    params = {
    "query": venue,
    "ll": coordinates,
    "limit": 10,
    "radius": 10000}
    #initiate the request
    response = requests.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        results = data.get("results", [])
        #extract relevant fields
        places = []
        for place in results:
            name = place.get("name", "N/A")
            latitude = place.get("geocodes", {}).get("main", {}).get("latitude", "N/A")
            longitude = place.get("geocodes", {}).get("main", {}).get("longitude", "N/A")
            places.append({"Name": name, "Latitude": latitude, "Longitude": longitude})

        #return the results as ddataframe
        return pd.DataFrame(shops)
    else:
        print(f"Error: {response.status_code}, Message: {response.text}")
        return pd.DataFrame()

In [243]:
shops = foursquare_places("coffee_shops", "50.9290,6.9421")
shops

,Name,Latitude,Longitude
0,The Coffee Gang,50.930276,6.940809
1,Gustafson's Coffee Bar,50.930794,6.939413
2,Cafecafe - Urban Eats & Coffee,50.936329,6.933461
3,Blooming Coffeebar and Roastery,50.934228,6.952436
4,Coffee Lounge,50.937741,6.945384
5,The Coffee Gang,50.953182,6.957026
6,Jaely's Coffee,50.947523,6.922620
7,Jules Coffee,50.916965,6.924155
8,Pico Coffee,50.954344,6.957774
9,CoffeeBrothers,50.928754,6.943356


Export results into data folder as a json: data/coffe_shops.json

In [247]:
import os
import json
import pandas as pd

# Output file path
output_file = "data/coffee_shops.json"
# Ensure the directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)
# Convert the DataFrame to a list of dictionaries
shops_list = shops.to_dict(orient="records")
# Write the data to the JSON file
with open(output_file, "w") as json_file:
    json.dump(shops_list, json_file, indent=4)
print(f"Data saved to {output_file}")

Data saved to data/coffee_shops.json


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here